# ขั้นที่ 3: หางานที่ตรงกับความต้องการของผู้สมัครมากที่สุด
ในขั้นตอนนี้เราสร้าง raw text ที่รวบรวมความต้องการของผู้สมัคร และของบริษัท จากนั้นแปลงเป็น vector ด้วย TF-IDF นำ vector ของผู้สมัครมาเปรียบเทียบกับ vector ของตำแหน่งงาน ผลที่ได้จะเป็นลำดับงานที่ตรงกับความต้องการของผู้สมัครมากที่สุดเรียงลำดับจากมากไปหาน้อย

## สร้าง vector ผู้สมัครและตำแหน่งงาน

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
#df = pd.read_csv("data/jobthai/df_noOutliers.csv", index_col='Unnamed: 0')
position = pd.read_csv("data/jobthai/position_df.csv")
applicant = pd.read_csv("data/jobthai/applicant_df.csv")

D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (5,6,22,26,43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
applicant = applicant[~applicant['WantedPosition_1'].isnull()]

## vector ผู้สมัคร 

In [4]:
WantedPosition1 = applicant['WantedPosition_1'].fillna('')
Duty1 = applicant['PrevDuty_1'].fillna('')
Duty2 = applicant['PrevDuty_2'].fillna('')
Duty3 = applicant['PrevDuty_3'].fillna('')

app = WantedPosition1 + ' ' + Duty1 + ' ' + Duty2 + ' ' + Duty3

## vector ตำแหน่งงาน

In [5]:
Title = position['Title'].fillna('')
Description = position['Description'].fillna('')

pos = Title + ' ' + Description

In [6]:
#Import Library สำหรับสร้าง tfidf model
#Import Dictionary
from gensim.corpora.dictionary import Dictionary
#Import TfidfModel
from gensim.models.tfidfmodel import TfidfModel
import collections
import itertools
#Import library ใช้สำหรับการตัดและนับคำ
from collections import Counter
from pythainlp.tokenize import word_tokenize

D:\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [7]:
def create_token(groupTitle):
    #ใส่คอลัมน์ที่ต้องการดู
    tokens = []
    for i in range(0,groupTitle.shape[0]):
        text = "".join(groupTitle[i])

        #Preprocessing step
        punctuation = [',','/','-','&','','\r\r\n-','\r\n','\r','\n','\t']
        conjunction = ['และ']

        pattern = punctuation + conjunction
        escape_pattern = ['(',')']

        for word in escape_pattern:
            text = re.sub(re.escape(word), '', text)    
        for word in pattern:
            text = re.sub(word, '', text)
        #เอาเว้นวรรค ' ' ออกจากข้อความ   
        text = re.sub(' ', '', text)
        #ตัดและนับคำเพื่อหา keyword ของแต่ละกลุ่มอาชีพ
        tokens.append(word_tokenize(text))
        #bow_simple = Counter(tokens)
    
    return tokens

In [8]:
pos_token = create_token(pos)

In [9]:
def create_tfidf(tokens):
    # Create a Dictionary from the articles: dictionary
    dictionary = Dictionary(tokens)
    # Create a MmCorpus: corpus
    corpus = [dictionary.doc2bow(token) for token in tokens]

    #เลือก document
    #doc = corpus[0]

    # Create a new TfidfModel using the corpus: tfidf
    tfidf = TfidfModel(corpus)
    
    return tfidf, corpus, dictionary

In [10]:
tfidf, corpus, dictionary = create_tfidf(pos_token)

In [11]:
#ฟังก์ชันสำหรับทำนายผล
def predictJob(app):
    test = app
    t = word_tokenize(test)
    t = dictionary.doc2bow(t)
    t = tfidf[t]

    return position.loc[sims[t].argmax()][['Title','Description']]

In [12]:
import gensim.similarities
sims = gensim.similarities.Similarity('/data', tfidf[corpus], num_features=len(dictionary))
print(sims)
print(type(sims))

Similarity index with 59826 documents in 1 shards (stored under /data)
<class 'gensim.similarities.docsim.Similarity'>


In [13]:
pred = predictJob(app[8])

In [14]:
app_profile = applicant[['WantedPosition_1','PrevDuty_1','PrevDuty_2','PrevDuty_3']]
pred = pd.DataFrame()
for i in range(0,20):
    current_pred = predictJob(app[i])
    current_pred = pd.concat([app_profile.loc[i], current_pred])
    pred = pd.concat([pred, current_pred], axis=1)

In [15]:
#กลับ dataframe
pred = pred.T
#เรียง column ใหม่
pred = pred[['WantedPosition_1','PrevDuty_1','PrevDuty_2','PrevDuty_3','Title','Description']]
pred = pred.rename(columns = {'Title': 'predict_Title', 'Description': 'predict_Description'})

In [16]:
#Save เป็น file excel
writer = pd.ExcelWriter('predictMostLikelyJob20.xlsx')
pred.to_excel(writer,'Sheet1')
writer.save()

In [17]:
pred

,WantedPosition_1,PrevDuty_1,PrevDuty_2,PrevDuty_3,predict_Title,predict_Description
0,Asst. CEO/Asst GM/Asst. Plant MGR / Production...,Productivity Improvement 改善\r\r\nLine Balancin...,1. Yield capacity analysis \...,1. MRP schedule\r\r\n Whe...,พนักงาน it,ดูแลระบบเครือข่าย ระบบข้อมูล ซ่อมและแก้ไขปัญหา...
0,ผู้จัดการงานเร่งรัดหนี้สินติดตามหนี้สิน/บังคับคดี,ติดตามหนี้เช่าซื้อ ดำเนินการยึดเครื่องจักร ประ...,ขายสินค้าเฟอร์นิเจอร์ กลุ่มพลาสติก และสแตนเลส ...,เร่งรัดหนี้สิน เป็นผู้รับมอบอำนาจของบริษัทฯ เพ...,พนักงานจัดซื้อ,1. จัดหาซัพพลายเออร์ใหม่ๆ พร้อมทั้งเปรียบเทียบ...
0,บริหารการตลาด,ดูแลบริหารงานส่วนการขาย การบริการ วางแผนงาน ...,ตลาดเพชรอารี - คิดแผนงานการตลาด และรับผิดชอบใน...,บ. ดำเนินกิจการทางด้านร้านอาหาร โดยรับผิดชอบด้...,พนักงานจัดซื้อ,1. จัดหาซัพพลายเออร์ใหม่ๆ พร้อมทั้งเปรียบเทียบ...
0,Graphics Design,"สอนวิชาทัศนศิลป์, จิตรกรรม","Desing Catalogue,Web Design , Presentation ,Ed...","Design Sport Promote , vtr , title TV ,storyboard",พนักงานแผนก Design,NaN
0,PM/CM/PE Engineer,เซ็นต์ออกแบบ-ควบคุม ดำเนินการขออนุญาตก่อสร้าง ...,"Drawing Drawn, Construction Permission, BOQ, S...",เคลียร์-เขียนแบบ Shop และ Trainning CAD น้องๆว...,วิศวกรเขียนแบบ/ออกแบบ CAD 3D&2D,- เขียนแบบ CAD 2D และ 3D ตามแบบที่ได้รับมอบหมา...
0,Production Manager,-\tSupervise Production team .\r\r\n-\tEnsure ...,-\tSupervise Warehouse and Production team .\r...,-\tHandle the full function of Production Mana...,Production,NaN
0,ผู้จัดการฝ่ายบุคคล,รับผิดชอบดูแลการบริหารงานบุคคล ในธุรกิจรถยนต์ ...,1. งานวางแผนกำลังคนและสรรหาว่าจ้าง - กำหนดนโยบ...,รับผิดชอบในการบริหารงานฝ่ายทรัพยากรบุคคล ซึ่งค...,พนักงานจัดซื้อ,1. จัดหาซัพพลายเออร์ใหม่ๆ พร้อมทั้งเปรียบเทียบ...
0,ผู้จัดการฝ่ายจัดซื้อ,จัดซื้อ วัตถุ ดิบ จัดหา แหล่งวัตถุดิบ ลดต้นทุน...,1.Sytem iso 9001 and ts 16949.contro quality s...,"Planning, production, use grip to people.plans...",เจ้าหน้าที่ธุรการฝ่ายวิศวกรรม,ทำงานเอกสารส่วนงานช่าง จัดทำประวัติและทะเบียนเ...
0,วิศวกรฝ่ายขาย SALE ENGINEER,"ประสานงานฝ่ายขาย 4 ฝ่ายขายสด , ขายเครดิต , ขาย...",ศึกษาข้อมูลผลิตภัณฑ์บริษัทฯ ศึกษากระบวนการผล...,ศึกษาข้อมูลผลิตภัณฑ์บริษัทฯ ศึกษากระบวนการผล...,พนักงานจัดซื้อ,1. จัดหาซัพพลายเออร์ใหม่ๆ พร้อมทั้งเปรียบเทียบ...
0,Operation & Reservation Manager,* จัดทำแพคเกจการขาย โปรโมชั่นต่างๆเพื่อดึงดูดใ...,1.โต้ตอบอิเมลกับลูกค้า(เอเย่นต์) ทางต่างประเทศ...,NaN,Reservation,- รับจองทัวร์ทางโทรศัพท์\r\r\n- ติดต่อไกด์ / พ...


## ตำแหน่งงานที่เหมาะสมกับผู้สมัคร 5 อันดับแรก

In [18]:
#ฟังก์ชันสำหรับทำนายผล
def predictNumSimilarJob(app,num):
    test = app
    t = word_tokenize(test)
    t = dictionary.doc2bow(t)
    t = tfidf[t]

    return position.loc[sims[t].argsort()[-num:][::-1]][['Title','Description']]

In [19]:
predNumJob = pd.DataFrame()
current_pred = predictNumSimilarJob(app[3],5)
current_pred = pd.concat([app_profile.loc[3], current_pred], axis=1)

D:\Anaconda3\lib\site-packages\pandas\core\indexes\api.py:77: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)


In [20]:
current_pred

,3,Title,Description
WantedPosition_1,Graphics Design,NaN,NaN
PrevDuty_1,"สอนวิชาทัศนศิลป์, จิตรกรรม",NaN,NaN
PrevDuty_2,"Desing Catalogue,Web Design , Presentation ,Ed...",NaN,NaN
PrevDuty_3,"Design Sport Promote , vtr , title TV ,storyboard",NaN,NaN
44860,NaN,พนักงานแผนก Design,NaN
8898,NaN,พนักงานออกแบบ ( Design ),NaN
27913,NaN,Design,เขียนแบบ ออกแบบเครื่องจักร
57042,NaN,Design,"- ออกแบบ (Design) 3D , 2D\r\r\n- งานเกี่ยวกับอ..."
48690,NaN,Design,- ติดตามงานและสรุปงานที่ออกแบบกับลูกค้า\r\r\n-...


In [21]:
x = pd.DataFrame()
y = pd.DataFrame()
num = 5
for i in range(0,10):
    current_pred = predictNumSimilarJob(app[i],5)
    for j in current_pred.index:
        x = pd.concat([app_profile.loc[i], current_pred.loc[j]])
        y = pd.concat([y, x], axis=1)
y = y.T
predictNumJob = y[['WantedPosition_1','PrevDuty_1','PrevDuty_2','PrevDuty_3','Title','Description']]

In [22]:
#predictNumJob

In [23]:
def predictEachJobNumber(number):
    print(applicant['WantedPosition_1'].loc[number])
    print('\n')
    print(applicant['PrevDuty_1'].loc[number])
    print(applicant['PrevDuty_2'].loc[number])
    print(applicant['PrevDuty_3'].loc[number])
    return predictNumSimilarJob(app.loc[number],5)

In [24]:
predictEachJobNumber(3)

Graphics Design


สอนวิชาทัศนศิลป์, จิตรกรรม
Desing Catalogue,Web Design , Presentation ,Editor VDO
Design Sport Promote , vtr , title TV ,storyboard


,Title,Description
44860,พนักงานแผนก Design,NaN
8898,พนักงานออกแบบ ( Design ),NaN
27913,Design,เขียนแบบ ออกแบบเครื่องจักร
57042,Design,"- ออกแบบ (Design) 3D , 2D\r\r\n- งานเกี่ยวกับอ..."
48690,Design,- ติดตามงานและสรุปงานที่ออกแบบกับลูกค้า\r\r\n-...
